# Import data
Import data downloaded from lantmäteriet (https://geotorget.lantmateriet.se/geodataprodukter/markhojdmodell-nedladdning-api)

Data can be downloaded via api or via stac browser https://radiantearth.github.io/stac-browser/#/external/api.lantmateriet.se/stac-hojd/v1/


In [11]:

import matplotlib.pyplot as plt

#from utils import get_tif_as_np_array, get_tif_from_np_array, info
from whitebox_workflows import WbEnvironment, show

from storm_water_management import utils

filename_path = "/home/chris/repos/storm_temp/data/1m"
filename = "63950_3150_25.tif"


In [ ]:
from pyproj import Transformer

wbe = WbEnvironment()
wbe.verbose = True
wbe.working_directory = filename_path
dem = wbe.read_raster(filename)
out_configs = dem.configs

print(f"North: {dem.configs.north}")
print(f"South: {dem.configs.south}")
print(f"East: {dem.configs.east}")
print(f"West: {dem.configs.west}")
print(f"EPSG code: {dem.configs.epsg_code}")
utils.info(dem)



    # verkar vara EPSG:5845 i.e.,  Horizontal CRS: EPSG:3006 Vertical CRS: EPSG:5613
    # a little bit unclear if 3006 or 5845
transformer = Transformer.from_crs("EPSG:5845", "EPSG:4326", always_xy=True)
lower_lon, lower_lat  = transformer.transform(dem.configs.west, dem.configs.south)
upper_lon, upper_lat  = transformer.transform(dem.configs.east, dem.configs.north)

#transformer = Transformer.from_crs("EPSG:3007", "EPSG:4326", always_xy=True)
#lower_lon, lower_lat  = transformer.transform(317459.14, 6395003.08)
#upper_lon, upper_lat  = transformer.transform(317500.0, 6395103.42)


print("-------------------------------")
print(f"North: {upper_lat}")
print(f"South: {lower_lat}")
print(f"East: {upper_lon}")
print(f"West: {lower_lon}")
print(f"EPSG code: {dem.configs.epsg_code}")

#mladens: bounds: [11.897039, 57.660217, 11.940792, 57.681619]

print(f"bounds: [{lower_lon}, {lower_lat}, {upper_lon}, {upper_lat}]")

In [ ]:
"""Main function."""
wbe = WbEnvironment()
wbe.verbose = True
wbe.working_directory = filename_path
dem = wbe.read_raster(filename)

raster_as_array = utils.get_tif_as_np_array(filename_path, filename)
dem_from_array = utils.get_tif_from_np_array(dem, raster_as_array)
utils.info(dem_from_array)




In [ ]:
# Smooth DEM. Parameters need to be set to proper values.
# dem_smoothed = wbe.feature_preserving_smoothing(dem_from_array, filter_size=11, normal_diff_threshold=10.0, iterations=3)
dem_smoothed = dem_from_array
# Fill depressions
dem_no_deps = wbe.fill_depressions_planchon_and_darboux(
    dem_smoothed, flat_increment=0.001
)
# dem_no_deps = wbe.fill_depressions(dem_smoothed, flat_increment=0.001)
# dem_no_deps = wbe.fill_depressions_wang_and_liu(dem_smoothed, flat_increment=0.001)
depression_depth = wbe.raster_calculator(
    "('dem_no_deps'-'dem')", [dem_no_deps, dem_from_array]
)

# wbe.write_raster(depression_depth, filename + 'depression_depth.tif')

# Plot depression filling
fig, ax = plt.subplots()
ax = show(
    depression_depth,
    ax=ax,
    title="Depression Filling",
    figsize=(10, 7),
    colorbar_kwargs={"label": "Elevation (m)", "location": "right", "shrink": 0.5},
    zorder=1,
    vmin=0,
    vmax=1,
)
ax.legend()
plt.show()

In [ ]:
# Flow accumulation analysis
channel_threshold = 50000.0
flow_accum = wbe.qin_flow_accumulation(dem_no_deps, out_type='cells', convergence_threshold=channel_threshold, log_transform=True)
# wbe.write_raster(flow_accum, filename + 'flow_accum.tif'))


In [ ]:
fig, ax = plt.subplots()
ax = show(
    flow_accum,
    ax=ax,
    title="Flow",
    figsize=(10, 7),
    colorbar_kwargs={"label": "Flow", "location": "right", "shrink": 0.5},
    zorder=1,
)
ax.legend()
plt.show()

#write to files

In [17]:
wbe.write_raster(depression_depth, 'hogsbo_depression_depth.tif')

In [18]:
dem = wbe.read_raster('hogsbo_depression_depth.tif')

out_configs = dem.configs
dem_truncated = wbe.new_raster(out_configs)

for row in range(dem.configs.rows):
    for col in range(dem.configs.columns):
        dem_truncated[row, col] = min(1., dem[row, col])

wbe.write_raster(dem_truncated, 'hogsbo_depression_depth_mod.tif')



In [ ]:
import matplotlib.pyplot as plt
import rasterio

# Öppna GeoTIFF
with rasterio.open("/home/chris/repos/storm_temp/data/1m/hogsbo_depression_depth_mod.tif") as src:
    data = src.read(1)  # läs första bandet
    profile = src.profile  # metadata (t.ex. CRS, upplösning)

# Plotta
plt.imshow(data, cmap="viridis")
plt.colorbar(label="Pixelvärde")
plt.title("GeoTIFF - band 1")
plt.show()

In [ ]:
#Write to png
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from PIL import Image

# Läs in GeoTIFF
with rasterio.open("/home/chris/repos/storm_temp/data/1m/hogsbo_depression_depth_mod.tif") as src:
    data = src.read(1)

# Gör en mask där värden = 0 ska vara transparenta
mask = data < 0.1

# Normalisera data (0–1) för colormap
normed = (data - data.min()) / (data.max() - data.min())
normed[mask] = np.nan  # sätt 0-värden till NaN för att bli transparent

# Välj colormap från matplotlib
cmap = plt.get_cmap("viridis")  # t.ex. "viridis", "terrain", "plasma"

# Applicera colormap → RGBA (0–1 floats)
rgba = cmap(normed)

# Gör om till 8-bitars (0–255)
rgba = (rgba * 255).astype(np.uint8)

# Sätt alpha=0 där masken är True
rgba[..., 3] = np.where(mask, 0, 255)

# Spara som PNG
img = Image.fromarray(rgba, mode="RGBA")
img.save("output_colormap.png")